In [757]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
from sklearn.metrics import roc_auc_score
from nltk.tokenize import WordPunctTokenizer
from nltk.stem.snowball import EnglishStemmer
from IPython.display import HTML, display_html
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
%matplotlib inline

# PATHS

In [758]:
FOLDER_PATH = "./"

# LOAD DATA

In [759]:
train = pd.read_csv(f"{FOLDER_PATH}train.csv")
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [761]:
test = pd.read_csv(f"{FOLDER_PATH}test.csv")
test_labels = pd.read_csv(f"{FOLDER_PATH}test_labels.csv")

test = pd.merge(test, test_labels[test_labels["toxic"] != -1])
test.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0001ea8717f6de06,Thank you for understanding. I think very high...,0,0,0,0,0,0
1,000247e83dcc1211,:Dear god this site is horrible.,0,0,0,0,0,0
2,0002f87b16116a7f,"""::: Somebody will invariably try to add Relig...",0,0,0,0,0,0
3,0003e1cccfd5a40a,""" \n\n It says it right there that it IS a typ...",0,0,0,0,0,0
4,00059ace3e3e9a53,""" \n\n == Before adding a new product to the l...",0,0,0,0,0,0


In [762]:
pred_test = test.copy()

class_labels = train.columns.tolist()[2:]
print(f"Labels: {class_labels}")

Labels: ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']


# FEATURE EXTRACTION

In [625]:
stemmer = EnglishStemmer()
tokenizer = WordPunctTokenizer()

In [627]:
train['stemmed_comment_text'] = train['comment_text'].apply(
    lambda x: " ".join([stemmer.stem(word) for word in tokenizer.tokenize(x)]))

In [628]:
%%time
test['stemmed_comment_text'] = test['comment_text'].apply(
    lambda x: " ".join([stemmer.stem(word) for word in tokenizer.tokenize(x)]))

CPU times: user 1min 3s, sys: 466 ms, total: 1min 4s
Wall time: 1min 4s


In [680]:
word_vectorizer = TfidfVectorizer(min_df=2, max_df=0.9, strip_accents='unicode', sublinear_tf=True,
                                  stop_words='english', max_features=10000, token_pattern=r'\w{1,}')

In [681]:
train_features = word_vectorizer.fit_transform(train['stemmed_comment_text'])

In [682]:
%%time
test_features = word_vectorizer.transform(test['stemmed_comment_text'])

CPU times: user 4.05 s, sys: 75.6 ms, total: 4.13 s
Wall time: 3.81 s


# MODELING

In [683]:
models_dict = {}
for label in class_labels:
    y_true = train[label]
    model = LogisticRegression(C=1.0, solver='sag', random_state=42)
    model.fit(train_features, y_true)
    models_dict[label] = model

In [684]:
%%time
for label in class_labels:
    pred_test[label] = models_dict[label].predict_proba(test_features)[:, 1]

CPU times: user 27.8 ms, sys: 1.74 ms, total: 29.5 ms
Wall time: 27.9 ms


In [685]:
print(f"ROC-AUC = {roc_auc_score(test.values[:, 2:-1].astype(int), pred_test.values[:, 2:-1]):.4f}")

ROC-AUC = 0.9765


# MODEL INTERPRETATION

In [638]:
def draw_html(tokens_and_weights, cmap=plt.get_cmap("bwr"), display=True,
              token_template="""<span style="background-color: {color_hex}">{token}</span>""",
              font_style="font-size:14px;"):

    def get_color_hex(weight):
        rgba = cmap(1. / (1 + np.exp(weight)), bytes=True)
        return '#%02X%02X%02X' % rgba[:3]
    
    tokens_html = [token_template.format(token=token, color_hex=get_color_hex(weight))
                   for token, weight in tokens_and_weights]
       
    raw_html = """<p style="{}">{}</p>""".format(font_style, ' '.join(tokens_html))
    if display:
        display_html(HTML(raw_html))
        
    return raw_html

In [640]:
example = test["stemmed_comment_text"][63976]

for label, model in models_dict.items():
    tokens_and_weights = []
    for word in example.split():
        index = word_vectorizer.vocabulary_.get(word)
        tokens_and_weights.append([word, model.coef_[0][index] if index else 0])
    print(f"Label: {label}")
    draw_html([(tok, weight) for tok, weight in tokens_and_weights], font_style='font-size:20px;');

Label: toxic


mel gibson is a nazi bitch who make shitti movi . he has so much buttsex that his asshol is now big enough to be consid a countri .

Label: severe_toxic


mel gibson is a nazi bitch who make shitti movi . he has so much buttsex that his asshol is now big enough to be consid a countri .

Label: obscene


...

mel gibson is a nazi bitch who make shitti movi . he has so much buttsex that his asshol is now big enough to be consid a countri .

Label: insult


mel gibson is a nazi bitch who make shitti movi . he has so much buttsex that his asshol is now big enough to be consid a countri .

Label: identity_hate


mel gibson is a nazi bitch who make shitti movi . he has so much buttsex that his asshol is now big enough to be consid a countri .

# REPORT

Главной целью задания было построить очень маленькую и очень быструю модель. Однако никаких конкретных ограничений на точность работы, время работы и размер модели даны не были. Такая поставка задачи допускает введения собственных уточнений целей задания.

Изучив задание и данные, я поставил себе цель, сделать максимально быструю модель, при этом потеряв в качестве работы не более ~1% относительно модели, занимающей 1-ое место на лидерборде.

Обычно, первая модель, которую хочется попробовать в задачах классификации с числовыми данными - это логистическая регрессия. При правильном выделении признаков из текстов оказалось, что обычная логистическая регрессия прекрасно справляется с поставленной задачей.

Самые известные способы выделения признаков из текстов, это применение различных счётчиков (CountVectorizer, TfidfVectorizer) и использование эмбендингов (Word2Vec, GloVe, FastText). Однако матрицы эмбендингов занимают довольно много места. Например, предобученные эмбендинги из библиотеки gensim занимают около 300-1000мб в зависимости от корпуса текстов, на котором они были обучены, и размера полученных векторов. Поэтому я решил отказаться от использования эмбендингов для построения максимально легкой модели.

Практически во всех случаях TfidfVectorizer работает лучше, чем CountVectorizer. Данная задача оказалась не исключением.

Для построения максимально быстрой модели, я решил отказаться от использования каких-либо нейронных сетей. Однако очевидно, что для этой задачи прекрасно бы подошли какие-нибудь сети на основе LSTM, GRU, CNN и их различных комбинаций и модификаций. Но также очевидно, что любая такая сеть работала бы значительно дольше, чем Logistic Regression, но при этом, судя по LB соревнования, не одна бы из таких глубоких сеток не смогла бы превысить скор логистической регрессии более чем на ~1%. Победители соревнования используют стэкинг большого числа различных нейросеток и кучу различных эмбендингов, из-за чего их модель является крайне медленной и громоздкой.

В данном ноутбуке я оставил 3 модели логистической регрессии, обученные на немного отличающихся признаках. Модели отличаются качеством и скоростью работы. Я не могу выделить одну из них как лучшую, потому что всё зависит от предпочтений между скоростью и точностью работы.

* Модель, обученную на признаках из TF-IDF, посчитанных на текстах, к словам которых была применена операция стемминга (лемматизация работает хуже) представлена выше. Её скор ~0.9765, что всего на 1.2% ниже, чем у победителей соревнования(0.9885). При этом одно предсказание у модели занимает ~0.43ms на преобработанных данных и ~1.49ms на сырых данных.

* В дополнении №1 представлена модель, обученная на признаках из TF-IDF, посчитанных на сырых текстах. Её скор ~0.9664, что уже на 2.2% ниже, чем у победителей соревнования. Зато одно предсказание у модели занимает ~0.35ms на преобработанных данных и ~0.41ms на сырых данных.

* В дополнении №2 представлена модель, обученная на признаках из TF-IDF, посчитанных не только по словам, но и по символам. Считать TF-IDF по символам в данной задаче имеет смысл, так как, например, в текстах часто встречаются типичные для токсичных сообщений символы. Скор этой модели ~0.98, что всего лишь на 0.8% ниже, чем у победителей соревнования. Одно предсказание у модели занимает ~0.25ms на преобработанных данных и ~2.41ms на сырых данных.

1) Какая получилась точность у модели? __Лучшая точность у модели получилась: ~0.98 roc-auc__

Следующие вопросы более актуальны для более сложных моделей, например, нейронных сетей, но кое-что можно сделать и для логистической регрессии.

2) Какие есть способы ускорения/уменьшения модели? __Ускорить модель можно с помощью распараллеливания процесса подсчёта TF-IDF и вычисления самой логистической регрессии. Так как размер логистической регрессии пропорционален количеству признаков, то уменьшить её размер можно с помощью уменьшения количества признаков, например, с помощью различных методов понижения размерности таких как: PCA, SVD и тд.__

3) Как выбрать баланс между качеством и скоростью? __Выбор баланса между скоростью и качеством напрямую зависит от конкретной задачи. То есть от того, где будет применяться данная модель. Например, если нужно делать предсказания онлайн и с телефона, то нужно отдать предпочтение скорости, а если нужно предсказывать с компьютера, то качеству.__

4) Что лучше — тяжелая модель и потом ее оптимизировать или сразу легкая? Поясните почему. __Лучше использовать сразу легкую модель. У многих моделей есть так называемое "bottleneck", которое очень сложно оптимизировать и которое способствует наибольшему замедлению скорости работы модели. Так же заранее никогда не известно насколько сложной должна быть модель, чтобы справиться с поставленной задачей. Например, если в данной задаче сразу броситься писать глубокую нейросетку, можно так и не узнать, что это оверхед и простой логистическая регресии будет достаточно. Также для сложных моделей, имеющих множество зависимых друг от друга параметров, совсем не очевидно как правильно их оптимизировать. Например, для сверточных нейросетей на ICML 2019 вышла крутая статья (https://arxiv.org/abs/1905.11946), авторы которой показали, что предыдущие методы оптимизации сверточных нейросетей были неоптимальными из-за нетривиальной зависимости между глубиной, числом каналов и разрешением свертоных нейросетей (также они нашли эту зависимость).__

5) Опишите ваш подход. Чем он лучше других возможных подходов? Какие у него могут быть недостатки? __Мой подход является традиционным для любых задач машинного обучения - нужно начинать с простого и двигаться к сложному. Очевидным преимуществом такого подхода является большая вероятность найти простое, но эффективное решение. Очевидным недостатком явлется возможность потратить много времени впустую. Например, мне повезло и самое простое решение оказалось подходящим, но в то же время могло случиться так, что только глубокая нейронная сеть подошла бы для данной задачи. В таком случае я потратил бы много времени на поиски не существующего простого решения, хотя мог бы сразу броситься писать нейросеть.__

6) Что можно сделать, чтобы улучшить классификатор? __Сложно, конечно, улучшить классификатор, полностью построенный на логистической регрессии. Но всё же есть 3 пути, чтобы сделать это:__
* __Более тчательный подбор гиперпараметров логистической регрессии. Так как в моделе используется шесть независимых классификаторов, то и гиперпараметры можно подбирать для каждого классификатора отдельно.__
* __Выделение дополнительных качественных признаков из текстов. Если нельзя увеличить время предобработки данных, то можно попробовать более качественно подобрать гиперпараметры TfidfVectorizer и/или более качественно почистить сами тексты. Если же можно увеличивать время работы/размер модели, то можно выделить признаки с помощью разных методов, например, с помощью эмбендингов и объединить полученные признаки.__
* __Можно попробовать учесть зависимость между классами текстов. При изучении данных видно, что многие классы являются зависимыми. Например, если текст содержит угрозу, то, скорее всего, он также является токсичным. Учесть такую зависимость можно, например, добавляя ответы одного из классификаторов в качестве признаков для другого.__

Выбрав линейную модель в качестве классификатора, интерпретируемость появляется автоматически :) Веса линейной модели являются мерой значимости соответствующих им признаков. Модуль веса определяет значимость признака, а знак указывает на класс, к которому данный признак "оттягивает" модель. В визуализации выше, синие слова имеют отрицательные веса и "оттягивают" модель к "dirty" классу, а красные слова имею положительные веса и "оттягивают" модель к "clear" классу.
Вообще говоря, с помощью нехитрого трюка и визуализации выше можно добавить интерпретируемость в любую модель. Нехитрый трюк заключается в следующем: будем наблюдать как модель реагирует на входные возмущения и в зависимости от этих реакций, станет понятно, какое влияение имеет каждое слово. В качестве входных возмущений можно подавать на вход модели один и тот же текст, но каждый раз заменять одно из слов на "UNK", и смотреть как от этого меняется предсказание модели.

# Appendix №1

In [662]:
word_vectorizer = TfidfVectorizer(min_df=2, max_df=0.9, strip_accents='unicode', sublinear_tf=True,
                                  stop_words='english', max_features=10000, token_pattern=r'\w{1,}')

In [663]:
train_features = word_vectorizer.fit_transform(train['comment_text'])

In [664]:
%%time
test_features = word_vectorizer.transform(test['stemmed_comment_text'])

CPU times: user 3.84 s, sys: 64.5 ms, total: 3.91 s
Wall time: 3.58 s


In [665]:
models_dict = {}
for label in class_labels:
    y_true = train[label]
    model = LogisticRegression(C=1.0, solver='sag', random_state=42)
    model.fit(train_features, y_true)
    models_dict[label] = model

In [666]:
%%time
for label in class_labels:
    pred_test[label] = models_dict[label].predict_proba(test_features)[:, 1]

CPU times: user 23.1 ms, sys: 1.4 ms, total: 24.5 ms
Wall time: 22.8 ms


In [667]:
print(f"ROC-AUC = {roc_auc_score(test.values[:, 2:-1].astype(int), pred_test.values[:, 2:-1]):.4f}")

ROC-AUC = 0.9664


# Appendix №2

In [738]:
word_vectorizer = TfidfVectorizer(min_df=2, max_df=0.9, strip_accents='unicode', sublinear_tf=True,
                                  stop_words='english', max_features=20000, token_pattern=r'\w{1,}')

char_vectorizer = TfidfVectorizer(sublinear_tf=True, strip_accents='unicode', analyzer='char',
                                  stop_words='english', ngram_range=(2, 5), max_features=60000)

In [739]:
word_train_features = word_vectorizer.fit_transform(train['stemmed_comment_text'])
char_train_features = char_vectorizer.fit_transform(train['comment_text'])
train_features = sp.hstack([word_train_features, char_train_features])

In [740]:
%%time
word_test_features = word_vectorizer.transform(test['stemmed_comment_text'])
char_test_features = char_vectorizer.transform(test['comment_text'])
test_features = sp.hstack([word_test_features, char_test_features])

CPU times: user 1min 15s, sys: 2.16 s, total: 1min 17s
Wall time: 1min 14s


In [741]:
models_dict = {}
for label in class_labels:
    y_true = train[label]
    model = LogisticRegression(C=1.1, solver='sag', random_state=42)
    model.fit(train_features, y_true)
    models_dict[label] = model

In [756]:
%%time
for label in class_labels:
    pred_test[label] = models_dict[label].predict_proba(test_features)[:, 1]

CPU times: user 15.3 s, sys: 1.42 s, total: 16.7 s
Wall time: 16.7 s


In [743]:
print(f"ROC-AUC = {roc_auc_score(test.values[:, 2:-1].astype(int), pred_test.values[:, 2:-1]):.4f}")

ROC-AUC = 0.9800
